<a href="https://colab.research.google.com/github/DanielVilarinho/Scrapping_teste/blob/main/Scrapping_Apresentacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Lista de marcas
marcas = ["heinz", "hemmer", "quero"]

# Lista de keywords relacionadas
keywords = [
    "Ketchup", "Molho de tomate", "Mostarda", "Maionese", "Molho barbecue",
    "Molho de pimenta", "Extrato de tomate", "Catchup picante", "Molho para churrasco", "Tempero pronto"
]

# Criar DataFrame
dm_keyword = pd.DataFrame({"Keyword": keywords})

# Adicionar coluna de URL
dm_keyword["URL"] = dm_keyword["Keyword"].apply(lambda x: f"https://mercado.carrefour.com.br/busca/{x.replace(' ', '%20')}?count=24")

# Exibir o DataFrame
display(dm_keyword)

,Keyword,URL
0,Ketchup,https://mercado.carrefour.com.br/busca/Ketchup...
1,Molho de tomate,https://mercado.carrefour.com.br/busca/Molho%2...
2,Mostarda,https://mercado.carrefour.com.br/busca/Mostard...
3,Maionese,https://mercado.carrefour.com.br/busca/Maiones...
4,Molho barbecue,https://mercado.carrefour.com.br/busca/Molho%2...
5,Molho de pimenta,https://mercado.carrefour.com.br/busca/Molho%2...
6,Extrato de tomate,https://mercado.carrefour.com.br/busca/Extrato...
7,Catchup picante,https://mercado.carrefour.com.br/busca/Catchup...
8,Molho para churrasco,https://mercado.carrefour.com.br/busca/Molho%2...
9,Tempero pronto,https://mercado.carrefour.com.br/busca/Tempero...


In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

def get_response(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return None

# Função para processar o HTML e extrair informações
def parse_html(response):
    if response:
      soup = BeautifulSoup(response.text, "html.parser")

      # Extrair hrefs dos produtos
      produtos = soup.find_all("a", class_="border rounded-lg border-[#f2f2f2] p-2 cursor-pointer overflow-hidden hover:shadow-md undefined flex flex-col gap-4")
      data = []
      position = 1

      for produto in produtos:
        #Capturando nome e link do produto
        name_element = produto.find("h2", class_="text-xs leading-4 text-[#333] text-left my-1 truncate-text h-12")
        name = name_element.get_text(strip=True) if name_element else None
        href = produto.get("href")

        #Capturando se o produto é patrocinado
        sponsored = produto.find("p",class_="text-xs")
        if sponsored:
          sponsored = True
        else:
          sponsored = False

        #Capturando preço do produto
        price = produto.find("span",class_="text-base font-bold text-default-dark").get_text(strip=True)

        #Capturando imagem
        img = produto.find("img",class_= "max-h-40 object-contain w-full").get("src")

        if name and href:
          name = name.encode("latin1").decode("utf-8", "ignore")
          data.append({"Produto": name, "Link": f"https://mercado.carrefour.com.br{href}", "Position" : position,"Sponsored" : sponsored,"Price": price,"Image": img})
          position += 1

      return pd.DataFrame(data)
    return [], []

from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

def get_response(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return None

# Função para processar o HTML e extrair informações
def parse_html(response):
    if response:
        soup = BeautifulSoup(response.text, "html.parser")

        # Extrair hrefs dos produtos
        produtos = soup.find_all("a", class_="border rounded-lg border-[#f2f2f2] p-2 cursor-pointer overflow-hidden hover:shadow-md undefined flex flex-col gap-4")
        data = []
        position = 1

        for produto in produtos:
            # Capturando nome e link do produto
            name_element = produto.find("h2", class_="text-xs leading-4 text-[#333] text-left my-1 truncate-text h-12")
            name = name_element.get_text(strip=True) if name_element else None
            href = produto.get("href")

            # Capturando se o produto é patrocinado
            sponsored = produto.find("p", class_="text-xs") is not None

            # Capturando preço do produto
            price_element = produto.find("span", class_="text-base font-bold text-default-dark")
            price = price_element.get_text(strip=True) if price_element else "Preço não disponível"

            # Capturando imagem
            img_element = produto.find("img", class_="max-h-40 object-contain w-full")
            img = img_element.get("src") if img_element else "Imagem não disponível"

            if name and href:
                name = name.encode("latin1").decode("utf-8", "ignore")
                data.append({
                    "Produto": name,
                    "Link": f"https://mercado.carrefour.com.br{href}",
                    "Position": position,
                    "Sponsored": sponsored,
                    "Price": price,
                    "Image": img
                })
                position += 1

        return pd.DataFrame(data)
    return pd.DataFrame(columns=["Produto", "Link", "Position", "Sponsored", "Price", "Image"])

def fetch_product_details(row):
    url = row["Link"]
    try:
        response = get_response(url)
        if not response:
            return row.name, "Marca não disponível"
        soup = BeautifulSoup(response.text, "html.parser")

        brand_span = soup.find("span", class_="mr-1 text-text")
        brand = brand_span.find("a", class_="underline").get_text(strip=True) if brand_span else "Marca não disponível"
        brand = brand.encode("latin1").decode("utf-8", "ignore") if brand else "Marca não disponível"

        return row.name, brand

    except requests.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return row.name, "Marca não disponível"

def update_product_details(df):
    df.loc[:, "Brand"] = None

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(fetch_product_details, [row for _, row in df.iterrows()]))

    for index, brand in results:
        df.loc[index, "Brand"] = brand

    return df

In [4]:
ft_data = []

for index, row in dm_keyword.iterrows():
    keyword = row["Keyword"]
    url = row["URL"]

    # Fazer a requisição para obter a resposta
    response = get_response(url)

    # Processar o HTML para extrair os produtos
    df_products = parse_html(response)

    # Adicionar a keyword correspondente aos produtos coletados
    if not df_products.empty:
        df_products["Keyword"] = keyword
        ft_data.append(df_products)

# Concatenar todos os DataFrames em um único DataFrame de fatos
ft_monitor = pd.concat(ft_data, ignore_index=True) if ft_data else pd.DataFrame(columns=["Produto", "Link", "Keyword"])

# Exibir o DataFrame final
#display(ft_monitor)

In [5]:
#Adicionar informações individuais de cada produto, como a marca
ft_monitor_product = update_product_details(ft_monitor)

In [6]:
ft_monitor_product['ClientProduct'] = ft_monitor_product['Brand'].apply(lambda x: True if x.lower() in marcas else False )

In [7]:
display(ft_monitor_product)

,Produto,Link,Position,Sponsored,Price,Image,Keyword,Brand,ClientProduct
0,Ketchup Hellmann's Tradicional 380g,https://mercado.carrefour.com.br/ketchup-hellm...,1,False,"R$ 12,79",https://carrefourbrfood.vtexassets.com/arquivo...,Ketchup,Marca no disponvel,False
1,Ketchup Carrefour 400g,https://mercado.carrefour.com.br/ketchup-carre...,2,False,"R$ 9,49",https://carrefourbrfood.vtexassets.com/arquivo...,Ketchup,Carrefour,False
2,Ketchup Original Cepêra 400 g,https://mercado.carrefour.com.br/ketchup-origi...,3,False,"R$ 9,89",https://carrefourbrfood.vtexassets.com/arquivo...,Ketchup,Marca no disponvel,False
3,Ketchup Tradicional Cepêra Frasco 400g,https://mercado.carrefour.com.br/ketchup-tradi...,4,False,"R$ 6,79",https://carrefourbrfood.vtexassets.com/arquivo...,Ketchup,Cepêra,False
4,Ketchup Heinz Tradicional 397 g,https://mercado.carrefour.com.br/ketchup-heinz...,5,False,"R$ 17,48",https://carrefourbrfood.vtexassets.com/arquivo...,Ketchup,Marca no disponvel,False
...,...,...,...,...,...,...,...,...,...
189,"Tempero Pronto para Massas, Batatas e Arroz co...",https://mercado.carrefour.com.br/tempero-pront...,20,False,"R$ 5,65",https://carrefourbrfood.vtexassets.com/arquivo...,Tempero pronto,Marca no disponvel,False
190,Tempero Pronto para Frango com Sal Sazon 60g,https://mercado.carrefour.com.br/tempero-pront...,21,False,"R$ 6,05",https://carrefourbrfood.vtexassets.com/arquivo...,Tempero pronto,Sazon,False
191,Tempero Pronto para Carnes Carrefour 50g,https://mercado.carrefour.com.br/tempero-pront...,22,False,"R$ 5,39",https://carrefourbrfood.vtexassets.com/arquivo...,Tempero pronto,Carrefour,False
192,Tempero Pronto para Almôndegas com Sal e Pimen...,https://mercado.carrefour.com.br/tempero-pront...,23,False,"R$ 4,49",https://carrefourbrfood.vtexassets.com/arquivo...,Tempero pronto,Satis,False


In [8]:
view = (ft_monitor_product.groupby("Keyword")["ClientProduct"]
        .value_counts(normalize=True) * 100).round(2)

print(view)

Keyword               ClientProduct
Catchup picante       False             66.67
                      True              33.33
Extrato de tomate     False             84.62
                      True              15.38
Ketchup               False             70.83
                      True              29.17
Maionese              False             87.50
                      True              12.50
Molho barbecue        False             80.00
                      True              20.00
Molho de pimenta      False            100.00
Molho de tomate       False            100.00
Molho para churrasco  False            100.00
Mostarda              False             83.33
                      True              16.67
Tempero pronto        False            100.00
Name: proportion, dtype: float64
